In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score,
    mean_absolute_error, mean_squared_error
)

warnings.filterwarnings("ignore")

In [3]:
# ==================================================
# LOAD DATA
# ==================================================
FILE_PATH = r"E:\WOU_BITS INTERN\39 BUS SYSTEM\39 BUS SYSTEM_FAULT_CONVERTED.xlsx"
df = pd.read_excel(FILE_PATH)

In [5]:
df

,All calculations,Phase Voltage A/Terminal in kV,Phase Voltage B/Terminal in kV,Phase Voltage C/Terminal in kV,Phase Current A/Terminal in kA,Phase Current B/Terminal in kA,Phase Current C/Terminal in kA,Electrical Frequency/Terminal in Hz,Deviation of the El. Frequency/Terminal in Hz,"Voltage Phasor, Angle/Terminal in deg",Active Power A/Terminal in MW,Active Power B/Terminal in MW,Active Power C/Terminal in MW,Angle between Voltage and Current Phasor/Terminal in deg,Terminal/Bus,Fault,Primary,Fault Location,CCT,Fault Line
0,1.500000,290.806187,-185.084586,-105.721601,-0.267677,0.207917,0.059761,59.999949,-0.000051,-8.954070,-77.842176,-38.482139,-6.317993,-171.232484,1,Yes,R1,1,2.279,1_2
1,1.500025,184.702943,-116.039600,-68.663343,3.290846,-2.027778,-1.263068,59.999950,-0.000050,-8.423720,607.828985,235.302527,86.726500,-0.782455,1,Yes,R1,1,2.279,1_2
2,1.500059,7.221638,-2.582302,-4.639336,4.597575,-2.811142,-1.786433,765.225645,705.225645,9.338942,33.202021,7.259217,8.287863,16.671468,1,Yes,R1,1,2.279,1_2
3,1.500109,-171.991193,110.008918,61.982275,1.415555,-0.737603,-0.677951,4868.799013,4808.799013,170.841653,-243.462911,-81.142932,-42.020964,172.235368,1,Yes,R1,1,2.279,1_2
4,1.500159,-91.168975,56.409224,34.759751,-3.588099,2.425924,1.162175,2471.947446,2411.947446,172.193379,327.123332,136.844498,40.396918,3.687533,1,Yes,R1,1,2.279,1_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135131,1.549759,0.946685,-0.244284,-0.702402,1.613669,-2.213103,0.599434,58.611248,-1.388752,15.609842,1.527637,0.540625,-0.421044,60.789483,68,Yes,R68,99,1.626,28_29
135132,1.549809,0.941378,-0.226225,-0.715153,1.709769,-2.233881,0.524111,59.363797,-0.636203,16.691937,1.609539,0.505360,-0.374820,59.655023,68,Yes,R68,99,1.626,28_29
135133,1.549859,0.933712,-0.207111,-0.726601,1.805021,-2.252674,0.447652,60.686353,0.686353,17.808097,1.685370,0.466553,-0.325265,58.625945,68,Yes,R68,99,1.626,28_29
135134,1.549909,0.929295,-0.189689,-0.739606,1.899476,-2.269521,0.370045,59.640927,-0.359073,18.862816,1.765173,0.430502,-0.273688,57.602970,68,Yes,R68,99,1.626,28_29


In [ ]:
# ==================================================
# BASE FEATURES
# ==================================================
BASE_FEATURES = [
    "Phase Voltage A/Terminal in kV",
    "Phase Voltage B/Terminal in kV",
    "Phase Voltage C/Terminal in kV",
    "Phase Current A/Terminal in kA",
    "Phase Current B/Terminal in kA",
    "Phase Current C/Terminal in kA",
    "Terminal/Bus"
]

# ==================================================
# CCT BINNING
# ==================================================
cct_bins = [1.58, 1.69, 1.78, 2.00, 2.50]
cct_labels = ["1.58–1.69", "1.69–1.78", "1.78–2.00", "2.00–2.50"]

df["CCT_Class"] = pd.cut(
    df["CCT"],
    bins=cct_bins,
    labels=cct_labels,
    include_lowest=True
)

# ==================================================
# LABELS
# ==================================================
LABELS = ["Primary", "Fault Location", "Fault Line", "CCT_Class"]

# ==================================================
# CLEAN DATA
# ==================================================
df = df[BASE_FEATURES + LABELS].dropna()
df = pd.get_dummies(df, columns=["Terminal/Bus"], drop_first=True)

FEATURES = [c for c in df.columns if c not in LABELS]

# ==================================================
# SCALE FEATURES
# ==================================================
scaler = StandardScaler()
X = scaler.fit_transform(df[FEATURES])

print("\n🤖 AUTOMATIC MODEL SELECTION: SVM vs RF vs ENSEMBLE")
print("==================================================")

final_summary = {}

# ==================================================
# LOOP OVER LABELS
# ==================================================
for label in LABELS:

    print("\n-----------------------------------------------")
    print(f"LABEL: {label}")
    print("-----------------------------------------------")

    y = LabelEncoder().fit_transform(df[label])

    if len(np.unique(y)) < 2:
        print("❌ Only one class present — skipped")
        continue

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        random_state=42,
        stratify=y
    )

    # ---------------------------------------------
    # SVM MODEL
    # ---------------------------------------------
    svm = SVC(
        kernel="rbf",
        probability=True,
        class_weight="balanced",
        C=10,
        gamma="scale"
    )
    svm.fit(X_train, y_train)
    svm_prob = svm.predict_proba(X_test)
    svm_pred = np.argmax(svm_prob, axis=1)

    acc_svm = accuracy_score(y_test, svm_pred) * 100

    # ---------------------------------------------
    # RANDOM FOREST MODEL
    # ---------------------------------------------
    rf = RandomForestClassifier(
        n_estimators=300,
        class_weight="balanced",
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)
    rf_prob = rf.predict_proba(X_test)
    rf_pred = np.argmax(rf_prob, axis=1)

    acc_rf = accuracy_score(y_test, rf_pred) * 100

    # ---------------------------------------------
    # ENSEMBLE (AUTO α SEARCH)
    # ---------------------------------------------
    best_alpha = None
    best_ens_acc = 0
    best_ens_prob = None
    best_ens_pred = None

    for alpha in np.arange(0.1, 1.0, 0.1):
        ens_prob = alpha * svm_prob + (1 - alpha) * rf_prob
        ens_pred = np.argmax(ens_prob, axis=1)
        acc = accuracy_score(y_test, ens_pred) * 100

        if acc > best_ens_acc:
            best_ens_acc = acc
            best_alpha = alpha
            best_ens_prob = ens_prob
            best_ens_pred = ens_pred

    # ---------------------------------------------
    # SELECT BEST MODEL
    # ---------------------------------------------
    scores = {
        "SVM": acc_svm,
        "Random Forest": acc_rf,
        "Ensemble": best_ens_acc
    }

    best_model = max(scores, key=scores.get)

    if best_model == "SVM":
        y_pred, y_prob = svm_pred, svm_prob
    elif best_model == "Random Forest":
        y_pred, y_prob = rf_pred, rf_prob
    else:
        y_pred, y_prob = best_ens_pred, best_ens_prob

    # ---------------------------------------------
    # FINAL METRICS
    # ---------------------------------------------
    acc = accuracy_score(y_test, y_pred) * 100
    prec = precision_score(y_test, y_pred, average="weighted") * 100
    rec = recall_score(y_test, y_pred, average="weighted") * 100
    f1 = f1_score(y_test, y_pred, average="weighted") * 100
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    try:
        roc_auc = roc_auc_score(y_test, y_prob, multi_class="ovr") * 100
    except:
        roc_auc = None

    # ---------------------------------------------
    # PRINT
    # ---------------------------------------------
    print(f"SVM Accuracy        : {acc_svm:.2f}%")
    print(f"RF Accuracy         : {acc_rf:.2f}%")
    print(f"Ensemble Accuracy   : {best_ens_acc:.2f}% (alpha={best_alpha})")
    print(f"✅ SELECTED MODEL   : {best_model}")

    print("\n--- FINAL METRICS ---")
    print(f"Accuracy (%)   : {acc:.2f}")
    print(f"Precision (%)  : {prec:.2f}")
    print(f"Recall (%)     : {rec:.2f}")
    print(f"F1-score (%)   : {f1:.2f}")
    print(f"ROC-AUC (%)    : {roc_auc:.2f}" if roc_auc else "ROC-AUC : Not defined")
    print(f"MAE            : {mae:.4f}")
    print(f"RMSE           : {rmse:.4f}")

    final_summary[label] = {
        "Best Model": best_model,
        "SVM Acc": acc_svm,
        "RF Acc": acc_rf,
        "ENS Acc": best_ens_acc
    }

# ==================================================
# SUMMARY
# ==================================================
print("\n🎯 FINAL MODEL SELECTION SUMMARY")
print("================================")
for lbl, res in final_summary.items():
    print(
        f"{lbl:15s} → {res['Best Model']:12s} | "
        f"SVM={res['SVM Acc']:.2f}% | "
        f"RF={res['RF Acc']:.2f}% | "
        f"ENS={res['ENS Acc']:.2f}%"
    )

print("\n✅ AUTOMATIC SVM + RF ENSEMBLE SELECTION COMPLETED")


🤖 AUTOMATIC MODEL SELECTION: SVM vs RF vs ENSEMBLE

-----------------------------------------------
LABEL: Primary
-----------------------------------------------
SVM Accuracy        : 100.00%
RF Accuracy         : 100.00%
Ensemble Accuracy   : 100.00% (alpha=0.1)
✅ SELECTED MODEL   : SVM

--- FINAL METRICS ---
Accuracy (%)   : 100.00
Precision (%)  : 100.00
Recall (%)     : 100.00
F1-score (%)   : 100.00
ROC-AUC (%)    : 100.00
MAE            : 0.0000
RMSE           : 0.0000

-----------------------------------------------
LABEL: Fault Location
-----------------------------------------------
